In [ ]:
!wget https://lodmedia.hb.bizmrg.com/cases/706486/CardioSpike_upd.zip
!wget https://lodmedia.hb.bizmrg.com/case_files/706486/test.csv

--2021-06-19 11:18:22--  https://lodmedia.hb.bizmrg.com/cases/706486/CardioSpike_upd.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3937389 (3.8M) [application/x-zip-compressed]
Saving to: ‘CardioSpike_upd.zip’

CardioSpike_upd.zip 100%[===================>]   3.75M  2.90MB/s    in 1.3s    

2021-06-19 11:18:24 (2.90 MB/s) - ‘CardioSpike_upd.zip’ saved [3937389/3937389]

--2021-06-19 11:18:24--  https://lodmedia.hb.bizmrg.com/case_files/706486/test.csv
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211758 (207K) [application/vnd.ms-excel]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>] 206.79K   376KB/s  

In [ ]:
!unzip -o CardioSpike_upd.zip

Archive:  CardioSpike_upd.zip
  inflating: CardioSpike.pdf         
  inflating: data/readme.md          
  inflating: data/train.csv          
  inflating: data/eval.py            
  inflating: data/sample_submission.csv  


In [ ]:
!cp data/eval.py eval.py 

In [ ]:
from eval import f1_score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('data/sample_submission.csv')

In [ ]:
def features(df):

  df['x-1'] = df['x'].diff(-1)
  df['x-2'] = df['x'].diff(-2)
  df['x-3'] = df['x'].diff(-3)
  df['x-4'] = df['x'].diff(-4)
  df['x-5'] = df['x'].diff(-5)
  df['x-6'] = df['x'].diff(-6)

  df['x+1'] = df['x'].diff(1)
  df['x+2'] = df['x'].diff(2)
  df['x+3'] = df['x'].diff(3)
  df['x+4'] = df['x'].diff(4)
  df['x+5'] = df['x'].diff(5)
  df['x+6'] = df['x'].diff(6)

  return df

In [ ]:
ids = list(data.id.unique())

In [ ]:
data2 = pd.DataFrame()
for id in ids:
    df = data[data.id == id]
    data2 = data2.append(features(df.copy()))

In [ ]:
test_ids = list(test.id.unique())

X_te = pd.DataFrame()
for id in test_ids:
    df = test[test.id == id]
    X_te = X_te.append(features(df.copy()))

In [ ]:
X_te.shape, test.shape

((15034, 15), (15034, 3))

In [ ]:
X_train = data2[data.id < 225]
X_val = data2[data.id >= 225]

In [ ]:
X_tr = X_train.drop(['y','id'], axis=1)
y_tr = X_train['y']
X_va = X_val.drop(['y','id'], axis=1)
y_va = X_val['y']

In [ ]:
categorical_features = [] # ['id']

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool, CatBoost

train_data = Pool(X_tr,
                  y_tr,
                 cat_features=categorical_features)

val_data = Pool(X_va,
                 y_va,
                 cat_features=categorical_features)

In [ ]:
model = CatBoostClassifier(iterations=2500,
                           #depth=6, 
                           loss_function='Logloss', 
                           eval_metric='F1',
                           logging_level='Verbose',
                           #bagging_temperature=0.8,
                           max_bin=128,
                           #random_strength=0.8,
                           #max_ctr_complexity=2,
                           #l2_leaf_reg=3,
                           od_type='Iter',
                           od_wait=100,
                           #task_type='GPU',
                           #used_ram_limit='15GB'                 
                          )

model.fit(train_data,
          eval_set=val_data,
          use_best_model=True,
         )

Learning rate set to 0.056497
0:	learn: 0.2608163	test: 0.2338747	best: 0.2338747 (0)	total: 25.4ms	remaining: 1m 3s
1:	learn: 0.0000000	test: 0.0000000	best: 0.2338747 (0)	total: 49.4ms	remaining: 1m 1s
2:	learn: 0.0000000	test: 0.0000000	best: 0.2338747 (0)	total: 73.2ms	remaining: 1m
3:	learn: 0.0000000	test: 0.0000000	best: 0.2338747 (0)	total: 97.1ms	remaining: 1m
4:	learn: 0.1108845	test: 0.0946815	best: 0.2338747 (0)	total: 125ms	remaining: 1m 2s
5:	learn: 0.0756149	test: 0.0621266	best: 0.2338747 (0)	total: 149ms	remaining: 1m 1s
6:	learn: 0.0650342	test: 0.0574850	best: 0.2338747 (0)	total: 173ms	remaining: 1m 1s
7:	learn: 0.0755753	test: 0.0621266	best: 0.2338747 (0)	total: 199ms	remaining: 1m 1s
8:	learn: 0.0738510	test: 0.0598086	best: 0.2338747 (0)	total: 224ms	remaining: 1m 1s
9:	learn: 0.0862898	test: 0.0803783	best: 0.2338747 (0)	total: 248ms	remaining: 1m 1s
10:	learn: 0.0926981	test: 0.0848556	best: 0.2338747 (0)	total: 272ms	remaining: 1m 1s
11:	learn: 0.1258883	test

In [ ]:
x = model.get_feature_importance(train_data)
for i in range(len(x)):
    print(X_tr.columns[i], x[i])

time 8.24402307780543
x 5.370332417353928
x-1 11.6571585563195
x-2 10.170415601865203
x-3 8.851066699225962
x-4 6.836039056999468
x-5 4.644680744924717
x-6 3.4177101172232365
x+1 10.03674082559642
x+2 10.560371334163882
x+3 8.40583882355856
x+4 5.723536980650538
x+5 3.673897561356395
x+6 2.4081882029567594


In [ ]:
pred = model.predict(X_va)

In [ ]:
prediction_proba = model.predict_proba(X_va)

In [ ]:
# подбираем порог

In [ ]:
import sklearn.metrics

for i in range(256):
  print(i/256, sklearn.metrics.f1_score(y_va, (prediction_proba[:,1] > i/256).astype(int)))

  #43359375 

0.0 0.3264853977844915
0.0 0.8401966810079902
0.00390625 0.4443834111507827
0.00390625 0.8401966810079902
0.0078125 0.46769052448565634
0.0078125 0.8401966810079902
0.01171875 0.48463855421686747
0.01171875 0.8401966810079902
0.015625 0.5017214397496088
0.015625 0.8401966810079902
0.01953125 0.5176890619928595
0.01953125 0.8401966810079902
0.0234375 0.5356662180349934
0.0234375 0.8401966810079902
0.02734375 0.5509708737864079
0.02734375 0.8401966810079902
0.03125 0.5675385166606951
0.03125 0.8401966810079902
0.03515625 0.5841949778434269
0.03515625 0.8401966810079902
0.0390625 0.6007228457295035
0.0390625 0.8401966810079902
0.04296875 0.6179819395367099
0.04296875 0.8401966810079902
0.046875 0.6374340235485181
0.046875 0.8401966810079902
0.05078125 0.6549560853199498
0.05078125 0.8401966810079902
0.0546875 0.673147548066537
0.0546875 0.8401966810079902
0.05859375 0.689853788214444
0.05859375 0.8401966810079902
0.0625 0.7066575778232219
0.0625 0.8401966810079902
0.06640625 0.71438434982

In [ ]:
pred.mean(), y_va.mean(), y_tr.mean()

(0.18678541340714888, 0.1950896618124925, 0.14067231400206984)

In [ ]:
#pred = model.predict(X_te)  

prediction_proba = model.predict_proba(X_te)  

pred = (prediction_proba[:,1] > .43359375).astype(int)


In [ ]:
pred.mean()

0.11221231874417986

In [ ]:
pred.shape, submit.shape

((15034,), (15034, 3))

In [ ]:
submit

,id,time,y
0,81,0,1
1,81,568,1
2,81,1140,1
3,81,1716,1
4,81,2284,1
...,...,...,...
15029,176,44332,1
15030,176,44968,1
15031,176,45596,1
15032,176,46904,1


In [ ]:
submit.y = pred

In [ ]:
!head data/sample_submission.csv

id,time,y
81,0,1
81,568,1
81,1140,1
81,1716,1
81,2284,1
81,2860,1
81,3420,1
81,3992,1
81,4564,1


In [ ]:
submit.to_csv('easycat_bin_433.csv', index=False)